In [2]:
%load_ext sql

# Get Old OCR

In [3]:
import os
from pprint import pprint

import numpy as np
import pandas as pd

## Figures from database

In [4]:
%sql postgresql:///pfocr20200224

In [20]:
%%sql old_ocr << SELECT pmcid, figure_number, result->'textAnnotations'->0->'locale' AS locale, result->'textAnnotations'->0->'description' AS description
FROM ocr_processors__figures
INNER JOIN ocr_processors ON ocr_processors__figures.ocr_processor_id = ocr_processors.id
INNER JOIN figures ON ocr_processors__figures.figure_id=figures.id
INNER JOIN papers ON figures.paper_id = papers.id
LIMIT 5;

 * postgresql:///pfocr20200224
5 rows affected.
Returning data to local variable old_ocr


In [21]:
old_ocr_df = old_ocr.DataFrame()

old_ocr_df.head()

,pmcid,figure_number,locale,description
0,PMC1539137,zek0080627210005,en,wt\n1 M NaCI\n0 5 15 30 60 120 180 Time (min)\...
1,PMC3094853,nihms254819f2,en,1b\n1e\n1k\nR-SNAC 1k\n1e\n1b\n1a\n1c\n1d\nMM\...
2,PMC3618260,1471-2164-14-190-5,en,"kDa\n0.2\n0,5\n1\n2\n3 6\n12\n24\nhpi\n100\nPS..."
3,PMC2290973,ehp0116-000524f2,en,TPSABI\nFRDI\nPLG\nVEGF\nNRID2\nNCAM1\nHLA-DOB...
4,PMC1919358,1471-213X-7-66-1,en,Workflow\nSteps:\nGene Ontology\nMicroarray\ng...


In [4]:
%%sql biothings_export << SELECT pmcid, figure_number,
word, symbol, xref AS entrez
FROM figures
INNER JOIN papers ON figures.paper_id = papers.id
INNER JOIN match_attempts ON figures.id = match_attempts.figure_id
LEFT JOIN symbols ON match_attempts.symbol_id = symbols.id
LEFT JOIN transformed_words ON match_attempts.transformed_word_id = transformed_words.id
LEFT JOIN lexicon ON match_attempts.symbol_id = lexicon.symbol_id
LEFT JOIN xrefs ON lexicon.xref_id = xrefs.id
WHERE xref IS NOT NULL;

 * postgresql:///pfocr20200224
1614452 rows affected.
Returning data to local variable biothings_export


In [5]:
bte_figures_df = biothings_export.DataFrame()
bte_figures_df["filename"] = bte_figures_df["figure_number"] + ".jpg"
bte_figures_df["figid"] = (
    bte_figures_df["pmcid"] + "__" + bte_figures_df["filename"]
)
bte_figures_df["url"] = (
    "https://www.ncbi.nlm.nih.gov/pmc/articles/"
    + bte_figures_df["pmcid"]
    + "/bin/"
    + bte_figures_df["filename"]
)

bte_figures = set(bte_figures_df["figid"])

print(len(bte_figures))
print(list(bte_figures)[0:5])
bte_figures_df.head()

100462
['PMC4500905__fnmol-08-00031-g0003.jpg', 'PMC3663255__18_2012_1151_Fig1_HTML.jpg', 'PMC2614970__1756-9966-27-83-6.jpg', 'PMC3487967__1471-2105-13-218-5.jpg', 'PMC5810575__PC_TPC2017RA00816DR1_f6.jpg']


,pmcid,figure_number,word,symbol,entrez,filename,figid,url
0,PMC6711159,nihms-1046734-f0001,TRAF6,TRAF6,7189,nihms-1046734-f0001.jpg,PMC6711159__nihms-1046734-f0001.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...
1,PMC3877363,pone.0085502.g004,c-MYC,c-Myc,4609,pone.0085502.g004.jpg,PMC3877363__pone.0085502.g004.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...
2,PMC2747168,zpq9990994580001,Trm5,TRM5,57570,zpq9990994580001.jpg,PMC2747168__zpq9990994580001.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
3,PMC2747168,zpq9990994580001,TYW1,TYW1,55253,zpq9990994580001.jpg,PMC2747168__zpq9990994580001.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,PMC2747168,zpq9990994580001,TYW2,TYW2,55039,zpq9990994580001.jpg,PMC2747168__zpq9990994580001.jpg,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...


In [14]:
bte_figures_df.to_csv(
    f"~/code/pathway-ocr/20191102_100k/pfocr_biothings.tsv",
    sep="\t",
    index=False,
)

In [15]:
os.system("jupyter nbconvert --to html biothings_export.ipynb")
os.system("mv biothings_export.html ~/code/pathway-ocr/20191102_100k/")

0